In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1,2'
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'

In [2]:
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer
device = "cuda"

import torch

In [11]:
import sys
sys.path.append('../')
from utils import load_data, replace_class_and_function_names, remove_docstrings
from metrics import accuracy_at_k

/home/sir_timio/.local/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [12]:
# !pip -qqq install bitsandbytes accelerate

In [8]:
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2", torch_dtype=torch.bfloat16, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [10]:
prompt = "My favourite condiment is"

model_inputs = tokenizer([prompt], return_tensors="pt").to(device)

generated_ids = model.generate(**model_inputs, max_new_tokens=100, do_sample=False)
tokenizer.batch_decode(generated_ids)[0]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'<s> My favourite condiment is soy sauce. I use it in cooking, as a dipping sauce, and even as a marinade. I love the savoury, umami flavour it adds to dishes.\n\nBut did you know that soy sauce is not just a seasoning? It also has some health benefits. Here are some reasons why you should include soy sauce in your diet:\n\n1. Rich in essential minerals: Soy sauce is a good source of essential minerals'

In [13]:
# settings :`cross_file_first`, `cross_file_random`, or `in_file`
settings = 'cross_file_first'
data = load_data('train', 'r', 'python', settings)

Loading data: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:05<00:00,  5.24s/it]


In [14]:
# raw_samples = np.random.choice(data['easy'], n_samples)
raw_samples = data['easy']

In [19]:
sample = raw_samples[1]

In [20]:
print(sample['next_line'])

    node = NodeFactory(host_identifier='foobar', enroll_secret='foobar')


In [25]:
next_line_prompt = "Using natural language, describe the following line of code: "

In [26]:
print(next_line_prompt + sample['next_line'].strip())

Using natural language, describe the following line of code: node = NodeFactory(host_identifier='foobar', enroll_secret='foobar')


In [27]:
model_inputs = tokenizer([next_line_prompt + sample['next_line'].strip()], return_tensors="pt").to(device)

In [30]:
generated_ids = model.generate(**model_inputs, max_new_tokens=100, do_sample=False)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [32]:
print(tokenizer.batch_decode(generated_ids)[0])

<s> Using natural language, describe the following line of code: node = NodeFactory(host_identifier='foobar', enroll_secret='foobar')

This line of code creates a new instance of a Node object, named 'node', using the NodeFactory class. The NodeFactory is likely a factory method or class responsible for creating new Node objects. The new Node object is initialized with two keyword arguments: 'host_identifier' and 'enroll_secret', both of which are set to the string value 'foobar'. These arguments are likely used to uniquely identify the node and provide a secret key for enrolling it into


In [34]:
print(sample['context'][0])

def create_app(config=ProdConfig):
    app = Flask(__name__)
    app.config.from_object(config)
    app.config.from_envvar('DOORMAN_SETTINGS', silent=True)

    register_blueprints(app)
    register_errorhandlers(app)
    register_loggers(app)
    register_extensions(app)
    register_auth_method(app)
    register_filters(app)

    return app


In [36]:
summary_prompt = "Summarize the following code, I will tip you 200 bucks: \n"

In [40]:
print(summary_prompt + sample['context'][0])

Summarize the following code, I will tip you 200 bucks: 
def create_app(config=ProdConfig):
    app = Flask(__name__)
    app.config.from_object(config)
    app.config.from_envvar('DOORMAN_SETTINGS', silent=True)

    register_blueprints(app)
    register_errorhandlers(app)
    register_loggers(app)
    register_extensions(app)
    register_auth_method(app)
    register_filters(app)

    return app


In [37]:
model_inputs = tokenizer([summary_prompt + sample['context'][0]], return_tensors="pt").to(device)

In [38]:
%%time
generated_ids = model.generate(**model_inputs, max_new_tokens=100, do_sample=False)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


CPU times: user 50.3 s, sys: 4.88 s, total: 55.2 s
Wall time: 55.1 s


In [39]:
print(tokenizer.batch_decode(generated_ids)[0])

<s> Summarize the following code, I will tip you 200 bucks: 
def create_app(config=ProdConfig):
    app = Flask(__name__)
    app.config.from_object(config)
    app.config.from_envvar('DOORMAN_SETTINGS', silent=True)

    register_blueprints(app)
    register_errorhandlers(app)
    register_loggers(app)
    register_extensions(app)
    register_auth_method(app)
    register_filters(app)

    return app

def register_blueprints(app):
    from . import auth, main, api
    app.register_blueprint(auth.bp)
    app.register_blueprint(main.bp)
    app.register_blueprint(api.bp)

def register_errorhandlers(app):
    from . import errors
    app.register_error_handler(400, errors.BadRequestError)
    app.


In [41]:
summary_prompt = "Please provide a summary in natural language of what the following code does:\n"

In [42]:
model_inputs = tokenizer([summary_prompt + sample['context'][0]], return_tensors="pt").to(device)
generated_ids = model.generate(**model_inputs, max_new_tokens=150, do_sample=False)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [43]:
print(tokenizer.batch_decode(generated_ids)[0])

<s> Please provide a summary in natural language of what the following code does:
def create_app(config=ProdConfig):
    app = Flask(__name__)
    app.config.from_object(config)
    app.config.from_envvar('DOORMAN_SETTINGS', silent=True)

    register_blueprints(app)
    register_errorhandlers(app)
    register_loggers(app)
    register_extensions(app)
    register_auth_method(app)
    register_filters(app)

    return app

This code defines a function called `create_app` that creates a new Flask web application. The function takes an optional argument `config`, which defaults to an instance of the `ProdConfig` class. The function sets the application's configuration by loading the settings from the given configuration object and from environment variables with the name `DOORMAN_SETTINGS`.

The function then registers various components of the application, including blueprints, error handlers, loggers, extensions, and an authentication method. It also registers filters that can be used

In [44]:
print(sample['code'])

# -*- coding: utf-8 -*-
"""Defines fixtures available to all tests."""





@pytest.yield_fixture(scope='function')
def app():
    """An application for the tests."""
    _app = create_app(config=TestConfig)
    ctx = _app.test_request_context()
    ctx.push()

    try:
        yield _app
    finally:
        ctx.pop()


@pytest.yield_fixture(scope='function')
def api():
    """An api instance for the tests, no manager"""
    # the mere presence of the env var should prevent the manage
    # blueprint from being registered
    os.environ['DOORMAN_NO_MANAGER'] = '1'

    _app = create_app(config=TestConfig)
    ctx = _app.test_request_context()
    ctx.push()

    try:
        yield _app
    finally:
        ctx.pop()


@pytest.fixture(scope='function')
def testapp(app):
    """A Webtest app."""
    return TestApp(app)


@pytest.fixture(scope='function')
def testapi(api):
    return TestApp(api)


@pytest.yield_fixture(scope='function')
def db(app):
    """A database for the tests."""
 

In [45]:
%%time
model_inputs = tokenizer([summary_prompt + sample['code']], return_tensors="pt").to(device)
generated_ids = model.generate(**model_inputs, max_new_tokens=150, do_sample=False)
print(tokenizer.batch_decode(generated_ids)[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Please provide a summary in natural language of what the following code does:
# -*- coding: utf-8 -*-
"""Defines fixtures available to all tests."""





@pytest.yield_fixture(scope='function')
def app():
    """An application for the tests."""
    _app = create_app(config=TestConfig)
    ctx = _app.test_request_context()
    ctx.push()

    try:
        yield _app
    finally:
        ctx.pop()


@pytest.yield_fixture(scope='function')
def api():
    """An api instance for the tests, no manager"""
    # the mere presence of the env var should prevent the manage
    # blueprint from being registered
    os.environ['DOORMAN_NO_MANAGER'] = '1'

    _app = create_app(config=TestConfig)
    ctx = _app.test_request_context()
    ctx.push()

    try:
        yield _app
    finally:
        ctx.pop()


@pytest.fixture(scope='function')
def testapp(app):
    """A Webtest app."""
    return TestApp(app)


@pytest.fixture(scope='function')
def testapi(api):
    return TestApp(api)


@pytest.

In [46]:
uncomplited_summary_prompt = "I need a plain language explanation of what this code does, not a continuation of the code. Please summarize the purpose and functionality of the following Python code:\n"

In [47]:
%%time
model_inputs = tokenizer([uncomplited_summary_prompt + sample['code']], return_tensors="pt").to(device)
generated_ids = model.generate(**model_inputs, max_new_tokens=150, do_sample=False)
print(tokenizer.batch_decode(generated_ids)[0])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> I need a plain language explanation of what this code does, not a continuation of the code. Please summarize the purpose and functionality of the following Python code:
# -*- coding: utf-8 -*-
"""Defines fixtures available to all tests."""





@pytest.yield_fixture(scope='function')
def app():
    """An application for the tests."""
    _app = create_app(config=TestConfig)
    ctx = _app.test_request_context()
    ctx.push()

    try:
        yield _app
    finally:
        ctx.pop()


@pytest.yield_fixture(scope='function')
def api():
    """An api instance for the tests, no manager"""
    # the mere presence of the env var should prevent the manage
    # blueprint from being registered
    os.environ['DOORMAN_NO_MANAGER'] = '1'

    _app = create_app(config=TestConfig)
    ctx = _app.test_request_context()
    ctx.push()

    try:
        yield _app
    finally:
        ctx.pop()


@pytest.fixture(scope='function')
def testapp(app):
    """A Webtest app."""
    return TestApp(ap